In [128]:
import pandas as pd
import seaborn as sns
import csv
import numpy as np
import sklearn
import pickle
import xgboost as xgb
from copy import deepcopy
from django.db.models import Model
from IPython.display import display
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from time import time
import pylab as plb
import matplotlib.pyplot as plt

%matplotlib inline

path = 'C:/Users/Admin/premier_predictions/Datasets/'

In [133]:
data_season_1 = pd.read_csv(path + '0708.csv')
data_season_2 = pd.read_csv(path + '0809.csv')
data_season_3 = pd.read_csv(path + '0910.csv')
data_season_4 = pd.read_csv(path + '1011.csv')
data_season_5 = pd.read_csv(path + '1112.csv')
data_season_6 = pd.read_csv(path + '1213.csv')
data_season_7 = pd.read_csv(path + '1314.csv')
data_season_8 = pd.read_csv(path + '1415.csv')
data_season_9 = pd.read_csv(path + '1516.csv')
data_season_10 = pd.read_csv(path + '1617.csv')
data_season_11 = pd.read_csv(path + '1718.csv')
data_season_12 = pd.read_csv(path + '1819.csv')
data_season_current = pd.read_csv(path + '1920.csv')


In [134]:
print(data_season_1.shape)
print(data_season_2.shape)
print(data_season_3.shape)
print(data_season_4.shape)
print(data_season_5.shape)
print(data_season_6.shape)
print(data_season_7.shape)
print(data_season_8.shape)
print(data_season_9.shape)
print(data_season_10.shape)
print(data_season_11.shape)
print(data_season_12.shape)
print(data_season_current.shape)

(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(380, 23)
(288, 23)


In [135]:
data_season_1.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,11/08/2007,Aston Villa,Liverpool,1,2,A,0,1,A,...,6,7,18,11,4,2,4,2,0,0
1,E0,11/08/2007,Bolton,Newcastle,1,3,A,0,3,A,...,9,5,15,16,4,3,1,1,0,0
2,E0,11/08/2007,Derby,Portsmouth,2,2,D,1,1,D,...,5,6,14,17,6,6,1,2,0,0
3,E0,11/08/2007,Everton,Wigan,2,1,H,1,0,H,...,8,4,8,13,6,2,0,0,0,0
4,E0,11/08/2007,Middlesbrough,Blackburn,1,2,A,1,0,H,...,6,4,16,16,13,3,3,4,0,0


In [136]:
data_season_1.Date = pd.to_datetime(data_season_1.Date, format='%d/%m/%Y')
data_season_2.Date = pd.to_datetime(data_season_2.Date, format='%d/%m/%Y')
data_season_3.Date = pd.to_datetime(data_season_3.Date, format='%d/%m/%Y')
data_season_4.Date = pd.to_datetime(data_season_4.Date, format='%d/%m/%Y')
data_season_5.Date = pd.to_datetime(data_season_5.Date, format='%d/%m/%Y')
data_season_6.Date = pd.to_datetime(data_season_6.Date, format='%d/%m/%Y')
data_season_7.Date = pd.to_datetime(data_season_7.Date, format='%d/%m/%Y')
data_season_8.Date = pd.to_datetime(data_season_8.Date, format='%d/%m/%Y')
data_season_9.Date = pd.to_datetime(data_season_9.Date, format='%d/%m/%Y')
data_season_10.Date = pd.to_datetime(data_season_10.Date, format='%d/%m/%Y')
data_season_11.Date = pd.to_datetime(data_season_11.Date, format='%d/%m/%Y')
data_season_12.Date = pd.to_datetime(data_season_12.Date, format='%d/%m/%Y')
data_season_current.Date = pd.to_datetime(data_season_current.Date, format='%d/%m/%Y')


In [137]:
data_season_9.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [138]:
cols = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']

data_season_1 = data_season_1[cols]
data_season_2 = data_season_2[cols]
data_season_3 = data_season_3[cols]
data_season_4 = data_season_4[cols]
data_season_5 = data_season_5[cols]
data_season_6 = data_season_6[cols]
data_season_7 = data_season_7[cols]
data_season_8 = data_season_8[cols]
data_season_9 = data_season_9[cols]
data_season_10 = data_season_10[cols]
data_season_11 = data_season_11[cols]
data_season_12 = data_season_12[cols]
data_season_current = data_season_current[cols]

In [139]:
data_season_9.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], dtype='object')

In [140]:
def total_goals_scored(data):
  teams = {key:[0] for key in data.HomeTeam.unique()}
  
  for i, row in data.iterrows():
    
    # add accumlated home team goals based on previous matches
    teams[row.HomeTeam].append(teams[row.HomeTeam][-1] + row.FTHG)
      
    # add accumlated away team goals based on previous matches
    teams[row.AwayTeam].append(teams[row.AwayTeam][-1] + row.FTAG)
  
  
  data['HTGS'] = 0
  data['ATGS'] = 0
  for i, row in data.iterrows():
    
    data.at[i, 'HTGS'] = teams[row.HomeTeam].pop(0)
    data.at[i, 'ATGS'] = teams[row.AwayTeam].pop(0)
  
  return data
data_season_1 = total_goals_scored(data_season_1)
data_season_2 = total_goals_scored(data_season_2)
data_season_3 = total_goals_scored(data_season_3)
data_season_4 = total_goals_scored(data_season_4)
data_season_5 = total_goals_scored(data_season_5)
data_season_6 = total_goals_scored(data_season_6)
data_season_7 = total_goals_scored(data_season_7)
data_season_8 = total_goals_scored(data_season_8)
data_season_9 = total_goals_scored(data_season_9)
data_season_10 = total_goals_scored(data_season_10)
data_season_11 = total_goals_scored(data_season_11)
data_season_12 = total_goals_scored(data_season_12)
data_season_current = total_goals_scored(data_season_current)

In [141]:
def total_goals_conceded(data):
  teams = {key:[0] for key in data.HomeTeam.unique()}
  
  for i, row in data.iterrows():
    
    # add accumlated home team goals based on previous matches
    teams[row.HomeTeam].append(teams[row.HomeTeam][-1] + row.FTAG)
      
    # add accumlated away team goals based on previous matches
    teams[row.AwayTeam].append(teams[row.AwayTeam][-1] + row.FTHG)
  
  
  data['HTGC'] = 0
  data['ATGC'] = 0
  for i, row in data.iterrows():
    
    data.at[i, 'HTGC'] = teams[row.HomeTeam].pop(0)
    data.at[i, 'ATGC'] = teams[row.AwayTeam].pop(0)
  
  return data

data_season_1 = total_goals_conceded(data_season_1)
data_season_2 = total_goals_conceded(data_season_2)
data_season_3 = total_goals_conceded(data_season_3)
data_season_4 = total_goals_conceded(data_season_4)
data_season_5 = total_goals_conceded(data_season_5)
data_season_6 = total_goals_conceded(data_season_6)
data_season_7 = total_goals_conceded(data_season_7)
data_season_8 = total_goals_conceded(data_season_8)
data_season_9 = total_goals_conceded(data_season_9)
data_season_10 = total_goals_conceded(data_season_10)
data_season_11 = total_goals_conceded(data_season_11)
data_season_12 = total_goals_conceded(data_season_12)
data_season_current = total_goals_conceded(data_season_current)


In [142]:
data_season_1.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC
375,2008-05-11,Portsmouth,Fulham,0,1,A,48,37,39,60
376,2008-05-11,Sunderland,Arsenal,0,1,A,36,73,58,31
377,2008-05-11,Tottenham,Liverpool,0,2,A,66,65,59,28
378,2008-05-11,West Ham,Aston Villa,2,2,D,40,69,48,49
379,2008-05-11,Wigan,Man United,0,2,A,34,78,49,22


In [143]:
def match_points(is_home_team, FTR):
  if FTR == 'H':
    return 3 if is_home_team else 0
  elif FTR == 'A':
    return 0 if is_home_team else 3
  else:
    return 1

def total_points(data):
  teams = {key:[0] for key in data.HomeTeam.unique()}
  
  for i, row in data.iterrows():
    
    # add accumlated home team goals based on previous matches
    teams[row.HomeTeam].append(teams[row.HomeTeam][-1] + match_points(is_home_team=True, FTR=row.FTR))
      
    # add accumlated away team goals based on previous matches
    teams[row.AwayTeam].append(teams[row.AwayTeam][-1] + match_points(is_home_team=False, FTR=row.FTR))
    
  data['HTP'] = 0
  data['ATP'] = 0
  for i, row in data.iterrows():
    
    data.at[i, 'HTP'] = teams[row.HomeTeam].pop(0)
    data.at[i, 'ATP'] = teams[row.AwayTeam].pop(0)
  
  return data

data_season_1 = total_points(data_season_1)
data_season_2 = total_points(data_season_2)
data_season_3 = total_points(data_season_3)
data_season_4 = total_points(data_season_4)
data_season_5 = total_points(data_season_5)
data_season_6 = total_points(data_season_6)
data_season_7 = total_points(data_season_7)
data_season_8 = total_points(data_season_8)
data_season_9 = total_points(data_season_9)
data_season_10 = total_points(data_season_10)
data_season_11 = total_points(data_season_11)
data_season_12 = total_points(data_season_12)
data_season_current = total_points(data_season_current)


In [144]:
data_season_12.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP
375,2019-05-12,Liverpool,Wolves,2,0,H,87,47,22,44,94,57
376,2019-05-12,Man United,Cardiff,0,2,A,65,32,52,69,66,31
377,2019-05-12,Southampton,Huddersfield,1,1,D,44,21,64,75,38,15
378,2019-05-12,Tottenham,Everton,2,2,D,65,52,37,44,70,53
379,2019-05-12,Watford,West Ham,1,4,A,51,48,55,54,50,49


In [145]:
def match_results(is_home_team, FTR):
  if FTR == 'H':
    return 'W' if is_home_team else 'L'
  elif FTR == 'A':
    return 'L' if is_home_team else 'W'
  else:
    return 'D'

def form_5_games(data):
  teams = {key:['M'] for key in data.HomeTeam.unique()} # 'M' as no matches have been played to win lose or draw
  
  for i, row in data.iterrows():
    
    # add accumlated home team goals based on previous matches
    teams[row.HomeTeam].append(match_results(is_home_team=True, FTR=row.FTR))
      
    # add accumlated away team goals based on previous matches
    teams[row.AwayTeam].append(match_results(is_home_team=False, FTR=row.FTR))
    
  
  teams2 = deepcopy(teams)
  teams3 = deepcopy(teams)
  teams4 = deepcopy(teams)
  teams5 = deepcopy(teams)   
  for key in teams:
    teams2[key].insert(0,'M')
    
    teams3[key].insert(0,'M')
    teams3[key].insert(0,'M')
    
    teams4[key].insert(0,'M')
    teams4[key].insert(0,'M')
    teams4[key].insert(0,'M')
    
    teams5[key].insert(0,'M')
    teams5[key].insert(0,'M')
    teams5[key].insert(0,'M')
    teams5[key].insert(0,'M')

    # initialize last 5 games to 'M'
  for i in range(1, 6):
    data['HM' + str(i)] = 'M'
    data['AM' + str(i)] = 'M'
  
  for i, row in data.iterrows():
    
    data.at[i, 'HM5'] = teams5[row.HomeTeam].pop(0)
    data.at[i, 'HM4'] = teams4[row.HomeTeam].pop(0)
    data.at[i, 'HM3'] = teams3[row.HomeTeam].pop(0)
    data.at[i, 'HM2'] = teams2[row.HomeTeam].pop(0)
    data.at[i, 'HM1'] = teams[row.HomeTeam].pop(0)
    
    data.at[i, 'AM5'] = teams5[row.AwayTeam].pop(0)
    data.at[i, 'AM4'] = teams4[row.AwayTeam].pop(0)
    data.at[i, 'AM3'] = teams3[row.AwayTeam].pop(0)
    data.at[i, 'AM2'] = teams2[row.AwayTeam].pop(0)
    data.at[i, 'AM1'] = teams[row.AwayTeam].pop(0)
  
  return data

data_season_1 = form_5_games(data_season_1)
data_season_2 = form_5_games(data_season_2)
data_season_3 = form_5_games(data_season_3)
data_season_4 = form_5_games(data_season_4)
data_season_5 = form_5_games(data_season_5)
data_season_6 = form_5_games(data_season_6)
data_season_7 = form_5_games(data_season_7)
data_season_8 = form_5_games(data_season_8)
data_season_9 = form_5_games(data_season_9)
data_season_10 = form_5_games(data_season_10)
data_season_11 = form_5_games(data_season_11)
data_season_12 = form_5_games(data_season_12)
data_season_current = form_5_games(data_season_current)


In [146]:
data_season_1.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM1,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5
0,2007-08-11,Aston Villa,Liverpool,1,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
1,2007-08-11,Bolton,Newcastle,1,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
2,2007-08-11,Derby,Portsmouth,2,2,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
3,2007-08-11,Everton,Wigan,2,1,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
4,2007-08-11,Middlesbrough,Blackburn,1,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M


In [147]:
data_season_9.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'AM1', 'HM2', 'AM2', 'HM3', 'AM3',
       'HM4', 'AM4', 'HM5', 'AM5'],
      dtype='object')

In [148]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 
        'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 
        'ATGC', 'HTP', 'ATP',
        'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 
        'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

data_season_1 = data_season_1[cols]
data_season_2 = data_season_2[cols]
data_season_3 = data_season_3[cols]
data_season_4 = data_season_4[cols]
data_season_5 = data_season_5[cols]
data_season_6 = data_season_6[cols]
data_season_7 = data_season_7[cols]
data_season_8 = data_season_8[cols]
data_season_9 = data_season_9[cols]
data_season_10 = data_season_10[cols]
data_season_11 = data_season_11[cols]
data_season_12 = data_season_12[cols]
data_season_current = data_season_current[cols]


In [149]:
prev_positions = pd.read_csv(path + "leaguepositions.csv")
prev_positions.set_index(['Team'], inplace=True)
prev_positions = prev_positions.fillna(18)

def last_season_pos(data, prev_positions, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = data.iloc[i].HomeTeam
        at = data.iloc[i].AwayTeam
        HomeTeamLP.append(prev_positions.loc[ht][year])
        AwayTeamLP.append(prev_positions.loc[at][year])
    data['HomeTeamLP'] = HomeTeamLP
    data['AwayTeamLP'] = AwayTeamLP
    return data

def most_recent_pos(data, prev_positions, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(288):
        ht = data.iloc[i].HomeTeam
        at = data.iloc[i].AwayTeam
        HomeTeamLP.append(prev_positions.loc[ht][year])
        AwayTeamLP.append(prev_positions.loc[at][year])
    data['HomeTeamLP'] = HomeTeamLP
    data['AwayTeamLP'] = AwayTeamLP
    return data

data_season_1 = last_season_pos(data_season_1, prev_positions, 0)
data_season_2 = last_season_pos(data_season_2, prev_positions, 1)
data_season_3 = last_season_pos(data_season_3, prev_positions, 2)
data_season_4 = last_season_pos(data_season_4, prev_positions, 3)
data_season_5 = last_season_pos(data_season_5, prev_positions, 4)
data_season_6 = last_season_pos(data_season_6, prev_positions, 5)
data_season_7 = last_season_pos(data_season_7, prev_positions, 6)
data_season_8 = last_season_pos(data_season_8, prev_positions, 7)
data_season_9 = last_season_pos(data_season_9, prev_positions, 8)
data_season_10 = last_season_pos(data_season_10, prev_positions, 9)
data_season_11 = last_season_pos(data_season_11, prev_positions, 10)
data_season_12 = last_season_pos(data_season_12, prev_positions, 11)
data_season_current = most_recent_pos(data_season_current, prev_positions, 12)

In [150]:
def mw(data):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    data['MW'] = MatchWeek
    return data

def mw_19_20(data):
    j = 1
    MatchWeek = []
    for i in range(288):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    data['MW'] = MatchWeek
    return data

data_season_1 = mw(data_season_1)
data_season_2 = mw(data_season_2)
data_season_3 = mw(data_season_3)
data_season_4 = mw(data_season_4)
data_season_5 = mw(data_season_5)
data_season_6 = mw(data_season_6)
data_season_7 = mw(data_season_7)
data_season_8 = mw(data_season_8)
data_season_9 = mw(data_season_9)
data_season_10 = mw(data_season_10)
data_season_11 = mw(data_season_11)
data_season_12 = mw(data_season_12)


data_season_current = mw_19_20(data_season_current)

In [159]:
all_seasons = pd.concat([data_season_1,
                         data_season_2,
                         data_season_3,
                         data_season_4,
                         data_season_5,
                         data_season_6,
                         data_season_8,
                         data_season_9,
                         data_season_10,
                         data_season_11,
                         data_season_12,
                         data_season_current],  ignore_index=True, sort=False)

all_seasons['HTFormPtsStr'] = all_seasons['HM1'] + all_seasons['HM2'] + all_seasons['HM3'] + all_seasons['HM4'] + all_seasons['HM5']
all_seasons['ATFormPtsStr'] = all_seasons['AM1'] + all_seasons['AM2'] + all_seasons['AM3'] + all_seasons['AM4'] + all_seasons['AM5']

def generate_points(result):
  if result == 'W':
    return 3
  elif result == 'D':
    return 1
  else:
    return 0

def sum_points(form):
  sum = 0
  for f in form:
    sum += generate_points(f)
  return sum
  
def form_points(data):
  
  data['HTFormPts'] = None
  data['ATFormPts'] = None
  for i, row in data.iterrows():
    data.at[i, 'HTFormPts'] = sum_points(row.HTFormPtsStr)
    data.at[i, 'ATFormPts'] = sum_points(row.ATFormPtsStr)
  
  return data

all_seasons = form_points(all_seasons)

In [161]:
all_seasons.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,AM3,AM4,AM5,HomeTeamLP,AwayTeamLP,MW,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts
4463,2020-03-07,Wolves,Brighton,0,0,D,41,32,34,40,...,D,D,L,18.0,15.0,29,WWDDL,LDDDL,8,3
4464,2020-03-07,Burnley,Tottenham,1,1,D,33,46,39,39,...,W,W,W,7.0,3.0,29,DWWDW,LLWWW,11,9
4465,2020-03-08,Chelsea,Everton,4,0,H,47,37,39,42,...,W,W,D,5.0,8.0,29,DWLDD,DLWWD,6,8
4466,2020-03-08,Man United,Man City,2,0,H,42,68,30,29,...,L,W,D,2.0,1.0,29,DWWDL,WWLWD,8,10
4467,2020-03-09,Leicester,Aston Villa,4,0,H,54,34,28,52,...,L,W,D,9.0,18.0,29,LLDDW,LLLWD,5,4


In [162]:
def streaks(data):
  
  data['HTWinStreak3'] = 0
  data['HTWinStreak5'] = 0
  data['HTLossStreak3'] = 0
  data['HTLossStreak5'] = 0

  data['ATWinStreak3'] = 0
  data['ATWinStreak5'] = 0
  data['ATLossStreak3'] = 0
  data['ATLossStreak5'] = 0
  
  for i, row in data.iterrows():
    
    if 'WWW' in row.HTFormPtsStr:
      data.at[i, 'HTWinStreak3'] = 1
    if 'WWWWW' in row.HTFormPtsStr:
      data.at[i, 'HTWinStreak5'] = 1
    
    if 'WWW' in row.ATFormPtsStr:
      data.at[i, 'ATWinStreak3'] = 1
    if 'WWWWW' in row.ATFormPtsStr:
      data.at[i, 'ATWinStreak5'] = 1
      
     
    if 'LLL' in row.HTFormPtsStr:
      data.at[i, 'HTLossStreak3'] = 1
    if 'LLLLL' in row.HTFormPtsStr:
      data.at[i, 'HTLossStreak5'] = 1
      
    if 'LLL' in row.ATFormPtsStr:
      data.at[i, 'ATLossStreak3'] = 1
    if 'LLLLL' in row.ATFormPtsStr:
      data.at[i, 'ATLossStreak5'] = 1
    
  return data

all_seasons = streaks(all_seasons)

In [163]:
all_seasons.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTFormPts,ATFormPts,HTWinStreak3,HTWinStreak5,HTLossStreak3,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5
0,2007-08-11,Aston Villa,Liverpool,1,2,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2007-08-11,Bolton,Newcastle,1,3,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2007-08-11,Derby,Portsmouth,2,2,D,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2007-08-11,Everton,Wigan,2,1,H,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2007-08-11,Middlesbrough,Blackburn,1,2,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
all_seasons['HTGD'] = all_seasons['HTGS'] - all_seasons['HTGC']
all_seasons['ATGD'] = all_seasons['ATGS'] - all_seasons['ATGC']

all_seasons['DiffPts'] = all_seasons['HTP'] - all_seasons['ATP']
all_seasons['DiffFormPts'] = all_seasons['HTFormPts'] - all_seasons['ATFormPts']

all_seasons['DiffLP'] = all_seasons['HomeTeamLP'] - all_seasons['AwayTeamLP']


In [165]:
all_seasons.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
4463,2020-03-07,Wolves,Brighton,0,0,D,41,32,34,40,...,0,0,0,0,0,7,-8,14,5,3.0
4464,2020-03-07,Burnley,Tottenham,1,1,D,33,46,39,39,...,0,1,0,0,0,-6,7,-2,2,4.0
4465,2020-03-08,Chelsea,Everton,4,0,H,47,37,39,42,...,0,0,0,0,0,8,-5,8,-2,-3.0
4466,2020-03-08,Man United,Man City,2,0,H,42,68,30,29,...,0,0,0,0,0,12,39,-15,-2,1.0
4467,2020-03-09,Leicester,Aston Villa,4,0,H,54,34,28,52,...,0,0,0,1,0,26,-18,25,1,-9.0


In [166]:
all_seasons.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP'],
      dtype='object')

In [167]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5',
       'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts', 'DiffLP']

all_seasons = all_seasons[cols]

In [168]:
all_seasons.to_csv(path + 'full_dataset2.csv')